In [ ]:
!wget  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?dl=0

In [ ]:
!tar -xvf "/content/genres.tar.gz?dl=0" -C "/content/sample_data"

In [3]:
!rm genres.tar.gz?dl=0

In [ ]:
!pip install pydub
!apt-get install ffmpeg
!pip install python_speech_features

In [5]:
from pydub import AudioSegment

In [6]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np
from python_speech_features import logfbank

In [ ]:
OUTPUT_DIR="/content/output" 
directory = "/content/sample_data/genres/"
i=0
for folder in os.listdir(directory):
  i+=1
  if i==11 :
    break
for file in os.listdir(directory+folder):
  print(file)
  au_audio = AudioSegment.from_file(directory+folder+'/'+file+'.au')
  wav_audio = au_audio.export(directory+folder+"/"+file+".wav", format="Wav")

function to get the distance between feature vectors and find neighbors:

In [10]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

Identify the nearest neighbors:

In [11]:
def nearestClass(neighbors):
    classVote = {}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

function for model evaluation

In [12]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [22]:
import warnings

Extract features from the dataset and dump these features into a binary .dat file “my.dat”:

In [ ]:
directory = "/content/sample_data/genres/"
f= open("my.dat" ,'wb')
i=0
for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break   
    for file in os.listdir(directory+folder):
      file = file.replace('.au','.wav')
      (rate,sig) = wav.read(directory+folder+"/"+file)
      mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
      covariance = np.cov(np.matrix.transpose(mfcc_feat))
      mean_matrix = mfcc_feat.mean(0)
      feature = (mean_matrix , covariance , i)
      pickle.dump(feature , f)
f.close()

Train and test split on the dataset:

In [ ]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  
    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  
trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)

Make prediction using KNN and get the accuracy on test data:

In [ ]:
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 5))) 
accuracy1 = getAccuracy(testSet , predictions)
print(accuracy1)

Testing the classifier with new audio file

In [ ]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np
from collections import defaultdict

In [ ]:
dataset = []
def loadDataset(filename):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
loadDataset("my.dat")


In [ ]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance


In [ ]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors  

In [ ]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]
results=defaultdict(int)
i=1
for folder in os.listdir("/content/sample_data/genres/"):
    results[i]=folder
    i+=1
(rate,sig)=wav.read("/content/sample_data/genres/blues/blues.00000.au")
mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature=(mean_matrix,covariance,0)
pred=nearestClass(getNeighbors(dataset ,feature , 5))
print(results[pred])

Exercise

In [ ]:
from collections import defaultdict
result = defaultdict(int)
directory = "/content/sample_data/genres/"
i=1
for file in os.listdir(directory):
  result[i]=file
  i+=1
print(result)


defaultdict(<class 'int'>, {1: 'blues', 2: 'reggae', 3: 'metal', 4: 'jazz', 5: 'classical', 6: 'rock', 7: 'pop', 8: 'hiphop', 9: 'country', 10: 'disco'})


In [ ]:
from pydub import AudioSegment

au_audio = AudioSegment.from_file('/content/sample_data/genres/blues/blues.00000.au')
wav_audio = au_audio.export("blues.00000.wav", format="Wav")
